In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from math import ceil
from tqdm import tqdm_notebook
import tqdm
from collections import Counter
from operator import itemgetter
from datetime import date, datetime

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import json

## RBC

In [ ]:
def get_article_rbc(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.text)
    heading = soup.h1.text
    try:
        lead = soup.find('div', 'article__text article__text_free').span.text
    except:
        lead = ''
    text = ' '.join([p.text for p in soup.find('div', 'article__text article__text_free').find_all('p')])
    return '. '.join([heading, lead, text]).replace('\xa0', ' ').strip()

In [ ]:
months = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
times = [['000000', '120000'], ['120000', '235959']]
rbc_20 = dict()
for month in tqdm_notebook(list(months.keys())[:2]):
    for day in tqdm_notebook(list(range(months[month]))):
        date = f'2020{month:02d}{day+1:02d}'
        rbc_20[f'2020.{month:02d}.{day+1:02d}'] = []
        for time in times:
            try:
                url = f'https://api.gdeltproject.org/api/v2/doc/doc?query=%20(domain:.rbc.ru%20OR%20domainis:rbc.ru)%20sourcelang:rus&mode=ArtList&maxrecords=250&sort=DateDesc&format=html&startdatetime={date}{time[0]}&enddatetime={date}{time[1]}'
#                 url = f'https://api.gdeltproject.org/api/v2/doc/doc?query=%20(domain:.mk.ru%20OR%20domainis:mk.ru)%20sourcelang:rus&mode=ArtList&maxrecords=250&sort=DateDesc&format=html&startdatetime={date}{time[0]}&enddatetime={date}{time[1]}'
    #             url = f'https://api.gdeltproject.org/api/v2/doc/doc?query=%20(domain:.kp.ru%20OR%20domainis:kp.ru)%20sourcelang:rus&mode=ArtList&maxrecords=250&sort=DateDesc&format=html&startdatetime={date}{time[0]}&enddatetime={date}{time[1]}'
                r = requests.get(url)
                soup = BeautifulSoup(r.text)
                rbc_20[f'2020.{month:02d}.{day+1:02d}'].extend([a['href'] for a in soup.body.table.find_all('a')])
                print(f'2020.{month:02d}.{day+1:02d}', len(soup.body.table.find_all('a')))
            except (AttributeError, IndexError):
                print(date)
with open('rbc_links_2020.json', 'w', encoding = 'utf-8') as f:
    json.dump(rbc_20, f)

In [ ]:
rbc_2020 = []
for date in tqdm_notebook(list(rbc_20.keys())):
    for link in tqdm_notebook(rbc_20[date]):
        if link not in pd.DataFrame(rbc_2020).Link:
            try:
                rbc_2020.append({'Media':'rbc.ru', 'Date':date, 'Link':link,
                                           'Text':get_article_rbc(link), 'Dep':'', 'Tags':[]})
            except (requests.exceptions.ConnectionError, AttributeError, 
                    IndexError, requests.exceptions.TooManyRedirects):
                print(link)
    if date.endswith('1'):
        pd.DataFrame(rbc_2020).to_csv('rbc_2020.csv', encoding='utf-8')
pd.DataFrame(rbc_2020).to_csv('rbc_2020.csv', encoding='utf-8')
pd.DataFrame(rbc_2020)

## RT

In [ ]:
def get_article_rt(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.text)
    heading = soup.h1.text
    subtitle = soup.find('div', 'article__summary article__summary_article-page js-mediator-article').text
    text = ' '.join([p.text for p in soup.find('div', 'article__text article__text_article-page js-mediator-article')
                     .find_all('p')])
    return '. '.join([heading, subtitle, text]).replace('\xa0', ' ')

In [ ]:
months = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
times = [['000000', '080000'], ['080000', '160000'],['160000', '235959']]
rt_20 = dict()
for month in tqdm_notebook(list(months.keys())):
    for day in tqdm_notebook(list(range(months[month]))):
        date = f'2020{month:02d}{day+1:02d}'
        rt_20[f'2020.{month:02d}.{day+1:02d}'] = []
        for time in times:
            try:
                url = f'https://api.gdeltproject.org/api/v2/doc/doc?query=%20(domain:.russian.rt.com%20OR%20domainis:russian.rt.com)%20sourcelang:rus&mode=ArtList&maxrecords=250&sort=DateDesc&format=html&startdatetime={date}{time[0]}&enddatetime={date}{time[1]}'
                r = requests.get(url)
                soup = BeautifulSoup(r.text)
                rt_20[f'2020.{month:02d}.{day+1:02d}'].extend([a['href'] for a in soup.body.table.find_all('a')])
                print(f'2020.{month:02d}.{day+1:02d}', len(soup.body.table.find_all('a')))
            except (AttributeError, IndexError):
                print(date)
with open('rt_links_2020.json', 'w', encoding = 'utf-8') as f:
    json.dump(rt_20, f)

In [ ]:
rt_2020 = []
for date in tqdm_notebook(list(rt_20.keys())):
    for link in tqdm_notebook(rt_20[date]):
        try:
            rt_2020.append({'Media':'russian.rt.com', 'Date':date, 'Link':link,
                                       'Text':get_article_rt(link), 'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects, ValueError):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(rt_2020).to_csv('rt_2020.csv', encoding='utf-8')
pd.DataFrame(rt_2020).to_csv('rt_2020.csv', encoding='utf-8')
pd.DataFrame(rt_2020)

## Gazeta.Ru

In [ ]:
def get_article_gazeta(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.text)
    try:
        heading = soup.find('div', 'h_1').text
        subtitle = soup.find('h1', 'sub_header').text
        lead = soup.find('span', 'intro').text
        text = ' '.join([p.text for p in soup.find('div', 'article_text').find_all('p')])
    except AttributeError:
        heading = soup.find('h1', 'article-text-header').text
        subtitle, lead = '', ''
        text = ' '.join([p.text for p in soup.find('div', 'article-text-body').find_all('p')])
    return '. '.join([heading, subtitle, lead, text])

In [ ]:
gazeta_links_2020 = set()
times = [['01.01.2020', '10.01.2020'], ['11.01.2020' ,'20.01.2020'], ['21.01.2020', '31.01.2020'],
        ['01.02.2020', '10.02.2020'], ['11.02.2020', '20.02.2020'], ['21.02.2020', '29.02.2020']]
for time in times:
    for text in ['%E7%E0%FF%E2%E8%EB', '%E3%EE%E2%EE%F0%E8%F2', '%E3%E0%E7%E5%F2%E0',
            '%F0%E0%F1%F1%EA%E0%E7%E0%EB', '%ED%E0%EF%E8%F1%E0%EB', '%F1%EE%EE%E1%F9%E8%EB',
            '%F1%EE%EE%E1%F9%E0%E5%F2', '%EE%E1%FA%FF%F1%ED%E8%EB', '%EF%EE%EF%E0%EB',
            '%E2%FB%F8%EB%E0', '%EF%E5%F0%E5%E4%E0%E5%F2', '%EF%F0%E8%F8%EB%E8']:
        page = 1
        while True:
            len_of_links = len(gazeta_links_2020)
    #     for text in ['заявил', 'говорит', 'считает', 'газета', 
    #                      'рассказал', 'написал', 'сообщил', 'сообщает', 
    #                      'объяснил', 'попал', 'вышла', 'передает', 'пришли']

#             time.sleep(1)
            url = f'https://www.gazeta.ru/search.shtml?p=page&tmp_p={page}&text={text}&article=&section=&from={time[0]}&to={time[1]}&sort_order=published_desc'
            r = requests.get(url)
            soup = BeautifulSoup(r.text)
            links_temp = ['https://www.gazeta.ru'+h2.a['href'] for h2 in soup.find_all('h2') 
                              if '2020' in h2.a['href']]
            print(time, text, page, len(links_temp))
            if links_temp:
                gazeta_links_2020.update(set(links_temp))
            else:
                break
            page += 1 

In [ ]:
gazeta_links_20 = defaultdict(set)

for link in tqdm_notebook(list(gazeta_links_2020)):
    if '/2020/' in link:
        index = link.index('/2020/')
        gazeta_links_20[link[index+1:index+11].replace('/', '.')].add(link)
        
gazeta_links_20_dict = dict()
for k,v in sorted(gazeta_links_20.items()):
    gazeta_links_20_dict[k] = list(v)
gazeta_links_20_dict

In [ ]:
gazeta_2020 = []
for date in tqdm_notebook(list(gazeta_links_20_dict.keys())):
    if 'a' not in date:
        for link in tqdm_notebook(gazeta_links_20_dict[date]):
    #         if link not in pd.DataFrame(gazeta_2020).Link:
            try:
                gazeta_2020.append({'Media':'gazeta.ru', 'Date':date, 'Link':link,
                                           'Text':get_article_gazeta(link), 'Dep':'', 'Tags':[]})
            except (requests.exceptions.ConnectionError, AttributeError, 
                    IndexError, requests.exceptions.TooManyRedirects):
                print(link)
        if date.endswith('1'):
            pd.DataFrame(gazeta_2020).to_csv('gazeta_2020.csv', encoding='utf-8')
pd.DataFrame(gazeta_2020).to_csv('gazeta_2020.csv', encoding='utf-8')
pd.DataFrame(gazeta_2020)

## Lenta.Ru

In [ ]:
def get_article_lenta(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.text)
    heading = soup.h1.text
    text = ' '.join([p.text for p in soup.find('div', 'js-topic__text').find_all('p')])
    return '. '.join([heading, text]).replace('\xa0', ' ')

In [ ]:
months = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
times = [['000000', '080000'], ['080000', '160000'],['160000', '235959']]
lenta_20 = dict()
for month in tqdm_notebook(list(months.keys())[:2]):
    for day in tqdm_notebook(list(range(months[month]))):
        date = f'2020{month:02d}{day+1:02d}'
        lenta_20[f'2020.{month:02d}.{day+1:02d}'] = []
        for time in times:
            try:
                url = f'https://api.gdeltproject.org/api/v2/doc/doc?query=%20(domain:.lenta.ru%20OR%20domainis:lenta.ru)%20sourcelang:rus&mode=ArtList&maxrecords=250&sort=DateDesc&format=html&startdatetime={date}{time[0]}&enddatetime={date}{time[1]}'
#                 url = f'https://api.gdeltproject.org/api/v2/doc/doc?query=%20(domain:.russian.rt.com%20OR%20domainis:russian.rt.com)%20sourcelang:rus&mode=ArtList&maxrecords=250&sort=DateDesc&format=html&startdatetime={date}{time[0]}&enddatetime={date}{time[1]}'
                r = requests.get(url)
                soup = BeautifulSoup(r.text)
                lenta_20[f'2020.{month:02d}.{day+1:02d}'].extend([a['href'] for a in soup.body.table.find_all('a')])
                print(f'2020.{month:02d}.{day+1:02d}', len(soup.body.table.find_all('a')))
            except (AttributeError, IndexError):
                print(date)
with open('lenta_links_2020.json', 'w', encoding = 'utf-8') as f:
    json.dump(lenta_20, f)

In [ ]:
lenta_2020 = []
for date in tqdm_notebook(list(lenta_20.keys())):
    for link in tqdm_notebook(lenta_20[date]):
#         if link not in pd.DataFrame(lenta_2020).Link:
        try:
            lenta_2020.append({'Media':'lenta.ru', 'Date':date, 'Link':link,
                                       'Text':get_article_lenta(link), 'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects, ValueError):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(lenta_2020).to_csv('lenta_2020.csv', encoding='utf-8')
pd.DataFrame(lenta_2020).to_csv('lenta_2020.csv', encoding='utf-8')
pd.DataFrame(lenta_2020)

## Fontanka

In [ ]:
def get_links_fontanka(date): #linke this '2020-05-04'
    page = 1
    links = []
    while True:
        link = f'https://www.fontanka.ru/cgi-bin/search.scgi?query=%D1%87%D1%82%D0%BE&rubric=&fdate={date}&tdate={date}&sortt=date&offset={page}'
        r = requests.get(link)
        soup = BS(r.text)
        links_temp = ['https://www.fontanka.ru'+li.find_all('a')[1]['href'] for li 
                 in soup.find_all('li') if li.find('div') and 'fontanka' not in li.find_all('a')[1]['href']]
        links.extend(links_temp)
        if links_temp:
            page += 1
        else:
            break
    return links

In [ ]:
fontanka_links_2020 = dict()
months = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

for month in tqdm_notebook(list(months.keys())[:2]):
    for day in tqdm_notebook(list(range(months[month]))):
        time.sleep(1)
        try:
            links = get_links_fontanka(f'2020-{month:0>2}-{day+1:0>2}')
            fontanka_links_2020[f'2020.{month:0>2}.{day+1:0>2}'] = links
            print(f'{day+1:0>2}.{month:0>2}.2020', len(links))
        except (AttributeError, IndexError):
            print(f'{day+1:0>2}.{month:0>2}.2020')
            
        if day%5 == 0:
            with open('fontanka_links_2020.json', 'w', encoding = 'utf-8') as f:
                json.dump(fontanka_links_2020, f)
with open('fontanka_links_2020.json', 'w', encoding = 'utf-8') as f:
    json.dump(fontanka_links_2020, f)
fontanka_links_2020

In [ ]:
fontanka_2020 = []
for date in tqdm_notebook(sorted(list(fontanka_links_20.keys()))):
    for link in tqdm_notebook(fontanka_links_20[date]):
        try:
            r = requests.get(link)
            soup = BS(r.text)
            fontanka_2020.append({'Media':'fontanka.ru', 'Date':date, 'Link':link,
                                  'Text':' '.join([p.text.strip() for p in soup.find('article').find_all('p') 
                                                   if '©' not in p.text]).replace('\xa0', ' '), 
                                  'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects, ValueError):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(fontanka_2020).to_csv('fontanka_2020.csv', encoding='utf-8')
pd.DataFrame(fontanka_2020).to_csv('fontanka_2020.csv', encoding='utf-8')
pd.DataFrame(fontanka_2020)

## 'Moscow' Agency

In [ ]:
months = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
moscow_20 = dict()
for month in tqdm_notebook(list(months.keys())[1:2]):
    for day in tqdm_notebook(list(range(months[month]))[8:]):
        date = f'{day+1:02d}.{month:02d}.2020'
        moscow_20[f'2020.{month:02d}.{day+1:02d}'] = []
        page = 1
        while True:
            try:
                url = f'https://www.mskagency.ru/?criteria=%D0%B8&from={date}&to={date}&page={page}&last_date={date}&rnd=1&list_only=1'
                r = requests.get(url)
                soup = BS(r.text)
                links_temp = ['https://www.mskagency.ru'+a['href'] for a in soup.find_all('a', 'js-title')]
                moscow_20[f'2020.{month:02d}.{day+1:02d}'].extend(links_temp)
            except (AttributeError, IndexError):
                print(date)
                
            if len(links_temp) < 50:
                print(f'2020.{month:02d}.{day+1:02d}', len(moscow_20[f'2020.{month:02d}.{day+1:02d}']))
                break
            else:
                page += 1
                
with open('moscow_links_2020.json', 'w', encoding = 'utf-8') as f:
    json.dump(moscow_20, f)

In [ ]:
moscow_2020 = []
for date in tqdm_notebook(sorted(list(moscow_20.keys()))):
    for link in tqdm_notebook(moscow_20[date]):
        try:
            r = requests.get(link)
            soup = BS(r.text)
            moscow_2020.append({'Media':'mskagency.ru', 'Date':date, 'Link':link,
                                  'Text':' '.join([soup.h1.text, ' '.join([p.text for p in soup.find('div', 'text').find_all('p')])]),
                                 'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects, ValueError):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(moscow_2020).to_csv('moscow_2020.csv', encoding='utf-8')
pd.DataFrame(moscow_2020).to_csv('moscow_2020.csv', encoding='utf-8')
pd.DataFrame(moscow_2020)

## Mediazona

In [ ]:
links = set()
for text in tqdm_notebook([
#     '%D1%81%D0%BA%D0%B0%D0%B7%D0%B0%D0%BB', 
                           '%D0%B3%D0%BE%D0%B2%D0%BE%D1%80%D0%B8%D1%82',
                          '%D1%81%D1%87%D0%B8%D1%82%D0%B0%D0%B5%D1%82', '%D1%80%D0%B0%D1%81%D1%81%D0%BA%D0%B0%D0%B7%D0%B0%D0%BB',
                          '%D0%BD%D0%B0%D0%BF%D0%B8%D1%81%D0%B0%D0%BB', '%D1%81%D0%BE%D0%BE%D0%B1%D1%89%D0%B8%D0%BB',
                          '%D0%BE%D0%B1%D1%8A%D1%8F%D1%81%D0%BD%D0%B8%D0%BB', '%D0%BF%D1%80%D0%B8%D1%88%D0%BB%D0%B8']):
    page = 1
    while True:
        time.sleep(0.5)
        link = f'https://zona.media/search?q={text}&page={page}'
        soup = BS(requests.get(link).text)
        links_temp = ['https://zona.media'+li.find('a')['href'] for li in soup.find_all('li', 'mz-search-result__item')]
        links.update(links_temp)
        print(page, len(links_temp), len(links))
        if links_temp and '/2018/' not in links_temp[-1]:
            page += 1
        else:
            break
            
with open('mediazona_links.json', 'w', encoding = 'utf-8') as f:
    json.dump({'data':list(links)}, f)

In [ ]:
links = defaultdict(list)
for link in tqdm_notebook(mediazona_links['data']):
    if '/2020/' in link:
        date = link[link.index('2020/'):link.index('2020/')+10].replace('/', '.')
    if '/2019/' in link:
        date = link[link.index('2019/'):link.index('2019/')+10].replace('/', '.')
    links[date].append(link)

In [ ]:
mediazona_19_20 = []
for date in tqdm_notebook(sorted(list(links.keys()))):
    for link in tqdm_notebook(links[date]):
        try:
            r = requests.get(link)
            soup = BS(r.text)
            mediazona_19_20.append({'Media':'zona.media', 'Date':date, 'Link':link,
                                  'Text':'. '.join([soup.find('header', 'mz-publish__title').text, 
                                                    ' '.join([p.text for p in 
                                                              soup.find('section', 'mz-publish__text').find_all('p')])]).replace('\xa0', ' '), 
                                  'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects, ValueError):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(mediazona_19_20).to_csv('mediazona_19_20.csv', encoding='utf-8')
pd.DataFrame(mediazona_19_20).to_csv('mediazona_19_20.csv', encoding='utf-8')
pd.DataFrame(mediazona_19_20)

## Meduza

In [ ]:
links = set()
for page in tqdm_notebook(range(30)):
    for text in ['заявил', 'говорит', 'считает', 'Медуза', 
                     'рассказал', 'написал', 'сообщил', 'сообщает', 
                     'объяснил', 'попал', 'вышла', 'передает', 'пришли']:
        time.sleep(1)
        links.update(set(['https://meduza.io/'+link for link in 
                   requests.get(f'https://meduza.io/api/w5/search?term={text}&page={page}&per_page=100&locale=ru').json()['collection']
                         if '2019' in link or '2020' in link]))

In [ ]:
meduza_links_2020 = defaultdict(set)

for link in tqdm_notebook(list(links)):
    if '/2020/' in link:
        index = link.index('/2020/')
        meduza_links_2020[link[index+1:index+11].replace('/', '.')].add(link)

In [ ]:
meduza_2020 = []
for date in tqdm_notebook(list(meduza_links_2020.keys())[27:]):
    for link in tqdm_notebook(meduza_links_2020[date]):
        if link not in pd.DataFrame(meduza_2020).Link:
            try:
                soup = BeautifulSoup(requests.get(link).text)
                title = soup.h1.text
                text = ' '.join([p.text for p in soup.find('div', 'GeneralMaterial-article') 
                                 if p.text and 'Напишите нам' not in p.text])

                meduza_2020.append({'Media':'meduza.io', 'Date':date, 'Link':link,
                                    'Text':' '.join([title, text]).replace('\xa0', ' '), 
                                    'Dep':'', 'Tags':[]})
            except (requests.exceptions.ConnectionError, AttributeError, IndexError, TypeError):
                print(link)
    if date.endswith('1'):
        pd.DataFrame(meduza_2020).to_csv('meduza_2020.csv', encoding='utf-8')
pd.DataFrame(meduza_2020).to_csv('meduza_2020.csv', encoding='utf-8')
pd.DataFrame(meduza_2020)

## The Bell

In [ ]:
thebell_links = defaultdict(list)
for page in tqdm_notebook(range(99, 410)):
    link = f'https://thebell.io/category/news/page/{page}/'
    r = requests.get(link)
    soup = BS(r.text)
    
    for div in soup.find_all('div', 'news-item-row'):
        if div.find('span'):
            date = div.find('span', 'news-item__info').text.split()
            date = f'{date[2]}.{months[date[1]]}.{date[0]}'
            thebell_links[date].append(div.find('a', 'news-item__link')['href'])

with open('thebell_links.json', 'w', encoding = 'utf-8') as f:
    json.dump(thebell_links, f)

In [ ]:
thebell_19_20 = []
for date in tqdm_notebook(sorted(list(thebell_links.keys()))):
    for link in tqdm_notebook(thebell_links[date]):
        try:
            r = requests.get(link)
            soup = BS(r.text)
            if len(date)==9:
                date = date[:-1]+'0'+date[-1]
            thebell_19_20.append({'Media':'thebell.io', 'Date':date, 'Link':link,
                                  'Text':'. '.join([soup.h1.text, ' '.join([p.text for p in soup.article.find_all('p') 
                                                                            if p.text and 'Фото:' not in p.text][:-1])]), 
                                  'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects, ValueError):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(thebell_19_20).to_csv('thebell_19_20.csv', encoding='utf-8')
pd.DataFrame(thebell_19_20).to_csv('thebell_19_20.csv', encoding='utf-8')
pd.DataFrame(thebell_19_20)